In [1]:
import baggianalysis as ba
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
def best_fit_transform(A, B):
    '''
    Calculates the least-squares best-fit transform that maps corresponding points A to B in m spatial dimensions
    Input:
      A: Nxm numpy array of corresponding points
      B: Nxm numpy array of corresponding points
    Returns:
      T: (m+1)x(m+1) homogeneous transformation matrix that maps A on to B
      R: mxm rotation matrix
      t: mx1 translation vector
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # translate points to their centroids
    centroid_A = np.mean(A, axis=0)
    centroid_B = np.mean(B, axis=0)
    AA = A - centroid_A
    BB = B - centroid_B

    # rotation matrix
    H = np.dot(AA.T, BB)
    U, S, Vt = np.linalg.svd(H)
    R = np.dot(Vt.T, U.T)

    # special reflection case
    if np.linalg.det(R) < 0:
       Vt[m-1,:] *= -1
       R = np.dot(Vt.T, U.T)

    # translation
    t = centroid_B.T - np.dot(R,centroid_A.T)

    # homogeneous transformation
    T = np.identity(m + 1)
    T[:m, :m] = R
    T[:m, m] = t

    return T, R, t


def nearest_neighbor(src, dst):
    '''
    Find the nearest (Euclidean) neighbor in dst for each point in src
    Input:
        src: Nxm array of points
        dst: Nxm array of points
    Output:
        distances: Euclidean distances of the nearest neighbor
        indices: dst indices of the nearest neighbor
    '''

    assert src.shape == dst.shape

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(dst)
    distances, indices = neigh.kneighbors(src, return_distance=True)
    return distances.ravel(), indices.ravel()


def icp(A, B, init_pose=None, max_iterations=20, tolerance=0.001):
    '''
    The Iterative Closest Point method: finds best-fit transform that maps points A on to points B
    Input:
        A: Nxm numpy array of source mD points
        B: Nxm numpy array of destination mD point
        init_pose: (m+1)x(m+1) homogeneous transformation
        max_iterations: exit algorithm after max_iterations
        tolerance: convergence criteria
    Output:
        T: final homogeneous transformation that maps A on to B
        distances: Euclidean distances (errors) of the nearest neighbor
        i: number of iterations to converge
    '''

    assert A.shape == B.shape

    # get number of dimensions
    m = A.shape[1]

    # make points homogeneous, copy them to maintain the originals
    src = np.ones((m+1,A.shape[0]))
    dst = np.ones((m+1,B.shape[0]))
    src[:m,:] = np.copy(A.T)
    dst[:m,:] = np.copy(B.T)

    # apply the initial pose estimation
    if init_pose is not None:
        src = np.dot(init_pose, src)

    prev_error = 0

    for i in range(max_iterations):
        # find the nearest neighbors between the current source and destination points
        distances, indices = nearest_neighbor(src[:m,:].T, dst[:m,:].T)

        # compute the transformation between the current source and nearest destination points
        T,_,_ = best_fit_transform(src[:m,:].T, dst[:m,indices].T)

        # update the current source
        src = np.dot(T, src)

        # check error
        mean_error = np.mean(distances)
        if np.abs(prev_error - mean_error) < tolerance:
            #break
            pass
        prev_error = mean_error

    # calculate final transformation
    T,_,_ = best_fit_transform(A, src[:m,:].T)

    return T, distances, i

In [3]:
# load the reference configuration
dest = np.loadtxt("points.dat")

In [4]:
arms = 30
N_per_arm = 20
# we want to retain the central bead
ids = [0, ]
# and the tips of the arms
ids += [a * N_per_arm + N_per_arm for a in range(arms)]
myfilter = ba.FilterByFunction(lambda p: p.index in ids)

In [5]:
parser = ba.GenericOxDNAParser("ba_topology.dat")
trajectory = ba.FullTrajectory(parser)
trajectory.add_filter(myfilter)
trajectory.initialise_from_trajectory_file("trajectory.dat")

In [6]:
# compute the average distance between the tips and the centre
trajectory.reset()
system = trajectory.next_frame()
dist_avg_per_tip = np.zeros(arms)
n_confs = 0
while system != None:
    n_confs += 1
    centre = system.particle_by_id(0).position
    for i, p_idx in enumerate(ids[1:]):
        distance = system.particle_by_id(p_idx).position - centre
        dist_avg_per_tip[i] += np.sqrt(np.dot(distance, distance))

    system = trajectory.next_frame()

dist_avg_per_tip /= n_confs
dist_avg = np.mean(dist_avg_per_tip)
print("Average distances:", dist_avg_per_tip)
print("Average distance:", dist_avg)
dest *= dist_avg

Average distances: [9.36684406 9.4940116  9.62017082 9.48213427 9.39944817 9.70235914
 9.45982576 9.45523218 9.5724091  9.46319756 9.60821982 9.50036672
 9.56528725 9.50893571 9.52288507 9.58172378 9.44829992 9.54437217
 9.52163335 9.48484047 9.56370912 9.57528692 9.64484665 9.5431361
 9.50580467 9.49789549 9.42946919 9.64134635 9.42184407 9.63967419]
Average distance: 9.52550698821793


In [7]:
trajectory.reset()
system = trajectory.next_frame()
src = []
for p in system.particles():
    if p.index == 0:
        centre = p.position
    else:
        src.append(p.position)
src -= centre

In [8]:
T, distances, iterations = icp(src, dest)

# make C a homogeneous representation of src
C = np.ones((arms, 4))
C[:,0:3] = np.copy(src)
# transform C according to the change of coordinates obtained with the icp procedure
C = np.dot(T, C.T).T
# obtain the new coordinates
src_corrected = C[:,:3]

with open("res.mgl", "w") as f:
    print(".Box:100,100,100", file=f)
    for p in dest:
        print("%lf %lf %lf @ 1 C[red]" % (p[0], p[1], p[2]), file=f)
        
    for p in src_corrected:
        print("%lf %lf %lf @ 1 C[blue]" % (p[0], p[1], p[2]), file=f)
    
    for p in src:
        print("%lf %lf %lf @ 1 C[green]" % (p[0], p[1], p[2]), file=f)


In [23]:
from scipy.optimize import linear_sum_assignment
from scipy.spatial import distance_matrix

# compute the distance matrix
graph_weights = distance_matrix(dest, src_corrected)
# perform the linear assignment
row_ind, col_ind = linear_sum_assignment(graph_weights)
# reorder the array according to the result of the linear assignment
src_corrected_reordered = src_corrected[col_ind,:]

with open("res_after_assignment.mgl", "w") as f:
    print(".Box:100,100,100", file=f)
    for i in range(arms):
        print("%lf %lf %lf @ 1 C[red]" % (dest[i][0], dest[i][1], dest[i][2]), file=f)
        #print("%lf %lf %lf @ 1 C[blue]" % (src_corrected[col_ind[i]][0], src_corrected[col_ind[i]][1], src_corrected[col_ind[i]][2]), file=f)
        print("%lf %lf %lf @ 1 C[blue]" % (src_corrected_reordered[i][0], src_corrected_reordered[i][1], src_corrected_reordered[i][2]), file=f)
        #print("%lf %lf %lf @ 1 C[blue]" % (dest[col_ind[i]][0], dest[col_ind[i]][1], dest[col_ind[i]][2]), file=f)
    

(30, 30)


In [52]:
# now we can compute the deformation gradient tensor F for each arm tip

# we first compute the matrices containing the difference between each point and all the others
# see here: https://stackoverflow.com/a/32473736/5140209
dest_diff_matrix = dest[:, np.newaxis, :] - dest
src_diff_matrix = src_corrected_reordered[:, np.newaxis, :] - src_corrected_reordered

# now we compute D for each point
# first we perform an outer product on the last axis
D = dest_diff_matrix[:,:,:,np.newaxis] * dest_diff_matrix[:,:,np.newaxis,:]
# then we sum over all the points (i.e. on the second axis)
D = np.sum(D, axis=1)
# and invert the matrices
D_inv = np.linalg.inv(D)

# we then compute A for each point
A = src_diff_matrix[:,:,:,np.newaxis] * dest_diff_matrix[:,:,np.newaxis,:]
# then we sum over all the points (i.e. on the second axis)
A = np.sum(A, axis=1)

F = A @ D_inv

# and now we can compute the Cauchy-Green strain tensor
F_T = np.transpose(F, axes=(0,2,1))
C = np.matmul(F_T, F)

# now we compute J and I
J = np.sqrt(np.linalg.det(C))
I = np.trace(C, axis1=1, axis2=2) * J**(-2./3.)


[3.29286187 3.12381192 3.38008265 3.08079144 3.09122441 3.18632558
 3.11207891 3.85507789 3.15196965 3.14373343 3.18649627 3.24800484
 3.25454155 3.11170803 3.10216032 3.05955865 3.07290418 3.23202422
 3.18006562 3.05228603 3.104292   3.19932891 3.06687846 3.12652611
 3.10019881 3.16425886 3.05086076 3.18821979 3.06208681 3.04724407]
